In [1]:
import numpy as np # Библиотека работы с массивами
import pandas as pd # Библиотека pandas

In [2]:
# Загружаем данные из файла csv
csv_url = (r'C:\Users\mihai\Neuron_Net\box_2NN.csv')
df = pd.read_csv(csv_url)
df.head()

,AcX,AcY,AcZ,Tmp,GyX,GyY,GyZ,work_notWork
0,540,144,-16660,27.07,-198,69,-149,0
1,21732,-5736,15908,27.07,-7947,32767,32767,0
2,11492,-10432,9136,27.17,-19892,-7853,-18745,0
3,16724,-756,-7196,27.07,194,-24127,6922,0
4,6352,5816,-8308,27.02,-2128,-10552,8214,0


In [3]:
dataset = df.values                 # Берем только значения массива(без индексов)
X = dataset[:,0:7].astype(float)   # Присваиваем им тип данных - float(с плавающей точкой) данным с 0 по 60 колонки
Y = dataset[:,7]                   # Присваеваем значению Y данные из столбца с индексом 60
Y = Y.astype(int)                   # Меняем тип данных столбца на 'int'(целочисленный тип данных)
print(X.shape)                      # Выводим размерность X
print(Y.shape)                      # Выводим размерность Y
print(Y)      

(1512, 7)
(1512,)
[0 0 0 ... 1 1 1]


### Создание обучающей, проверочной и тестовой выборки

In [4]:
# функция разделения на обучающую и проверочную/тестовую выборку
from sklearn.model_selection import train_test_split


# test_size=0.2 - будет выделено 20% от тренировочных данных 
# shuffle=True - перемешать данные
# x_train - данные для обучения
# x_test - данные для проверки
# y_train - правильные ответы для обучения
# y_test - правильные ответы для проверки
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)    
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(1209, 7)
(303, 7)
(1209,)
(303,)


### Создание нейронной сети

In [6]:
from tensorflow.keras.models import Sequential # НС прямого распространения
from tensorflow.keras.layers import Dense # Основные слои
from tensorflow.keras import utils # Утилиты для to_categorical
from tensorflow.keras.preprocessing import image # Для отрисовки изображения
from tensorflow.keras.optimizers import Adam # Алгоритмы оптимизации, для настройки скорости обучения

In [11]:
import mlflow
import mlflow.tensorflow 

In [34]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [13]:
# Создаем эксперимент и указываем хранилище артефактов
expname = "Keras_exp"
exp_id = mlflow.create_experiment(expname)

In [62]:
with mlflow.start_run(experiment_id=exp_id, run_name='Keras_autolog'):
    
    
    # Создаем сеть
    model = Sequential()

    # Добавляем слои
    model.add(Dense(70, input_dim=7, activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Компилируем сеть
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

    history = model.fit(
                    x_train, y_train, 
                    epochs=20, 
                    batch_size=128, 
                    validation_split=0.2,
                    verbose = 1)
    # Автоматом сохраняет model's parameters, metrics, artifacts
    mlflow.tensorflow.autolog()
    
    run_id = mlflow.active_run().info.run_id

C:\Users\mihai\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
8/8 [==============================] - 1s 52ms/step - loss: 895.1713 - accuracy: 0.4788 - val_loss: 616.1812 - val_accuracy: 0.5083
Epoch 2/20
8/8 [==============================] - 0s 12ms/step - loss: 504.5433 - accuracy: 0.4953 - val_loss: 374.0552 - val_accuracy: 0.4876
Epoch 3/20
8/8 [==============================] - 0s 12ms/step - loss: 341.5883 - accuracy: 0.5202 - val_loss: 246.7905 - val_accuracy: 0.5124
Epoch 4/20
8/8 [==============================] - 0s 10ms/step - loss: 235.8008 - accuracy: 0.5274 - val_loss: 208.0832 - val_accuracy: 0.5372
Epoch 5/20
8/8 [==============================] - 0s 23ms/step - loss: 187.4026 - accuracy: 0.4995 - val_loss: 208.4905 - val_accuracy: 0.5413
Epoch 6/20
8/8 [==============================] - 0s 13ms/step - loss: 152.1121 - accuracy: 0.5233 - val_loss: 176.6691 - val_accuracy: 0.5165
Epoch 7/20
8/8 [==============================] - 0s 12ms/step - loss: 126.6204 - accuracy: 0.5595 - val_loss: 157.7285 - val_accuracy: 0.5083

In [63]:
run_id

'f02c20e418924b43b7ed53528927166b'

In [66]:
# Запуск модели из млфлоу
keras_model = mlflow.keras.load_model("runs:/f02c20e418924b43b7ed53528927166b"+ "/model")
predictions = keras_model.predict(x_test)

In [67]:
predictions

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [2.9050258e-34],
       [1.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [1.4591780e-21],
       [5.0016183e-07],
       [1.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [2.0812235e-15],
       [1.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [3.2109043e-13],
       [0.000000